In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import mediapipe as mp
import cv2
import random
import os
from collections import Counter

In [2]:
# Загружаем модель
static_model = tf.keras.models.load_model('./models/static_model.h5')
dinamic_model = tf.keras.models.load_model('./models/dinamic_model.h5')

mp_hands = mp.solutions.hands # модель mediapipe hands для обнаружения ключевых точек
hands = mp_hands.Hands( 
    static_image_mode=False, 
    model_complexity=1, 
    min_detection_confidence=0.75, 
    min_tracking_confidence=0.75, 
    max_num_hands=2)

mp_pose = mp.solutions.pose 
pose = mp_pose.Pose( 
    static_image_mode=False,
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.75)

data_path = './slovo/data'
annotations_df = pd.read_csv(os.path.join('slovo','annotations.csv'), sep='\t')
actions_annotations_df = pd.read_csv(os.path.join('actions_annotations.csv'), sep=';')
actions = np.unique(np.array(actions_annotations_df['text']))

In [3]:
def extract_pose_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)

    return pose 

def pose_detection(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image, results


def pose_recognition(img):
    _, result = pose_detection(img)
    keypoints = extract_pose_keypoints(result)

    return keypoints

def dinamic_recognition(sequence):
    sequence = np.array(sequence)[:30].reshape(1, 30, -1)

    res = np.argmax(dinamic_model.predict(sequence))
    
    return actions[res]

In [4]:
def extract_keypoints(results):
    # в results.multi_hand_landmarks первые ключевые точки это правая рука, а вторые - левая
    right_hand_landmarks = None
    left_hand_landmarks = None

    if results.multi_handedness != None:
        if len(results.multi_handedness) == 2:
            right_hand_landmarks = results.multi_hand_landmarks[0]
            left_hand_landmarks = results.multi_hand_landmarks[1]
        else:
            if results.multi_handedness[0].classification[0].index == 1: #index right hand
                right_hand_landmarks = results.multi_hand_landmarks[0]
            else:
                left_hand_landmarks = results.multi_hand_landmarks[0]
            

    # 21 ориентир и 3 координаты у каждого x, y, z
    right_hand_np = np.array([[res.x, res.y, res.z] for res in right_hand_landmarks.landmark]).flatten() if right_hand_landmarks != None else np.zeros(21 * 3)
    left_hand_np = np.array([[res.x, res.y, res.z] for res in left_hand_landmarks.landmark]).flatten() if left_hand_landmarks != None else np.zeros(21 * 3)

    return np.concatenate([right_hand_np, left_hand_np]) # содержит ключевые точки элементов, которые представляют сглаженный массив значений x, y, z


def hand_detection(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image, results

def static_recognition(img):
    _, result = hand_detection(img)

    keypoints = extract_keypoints(result)

    if np.array_equal(keypoints, np.zeros(21 * 6)):
        return None

    # Изменяем форму массива на (1, 126)
    np_arr = keypoints.reshape(1, -1)

    return actions_annotations_df.at[np.argmax(static_model.predict(np_arr)), 'text']

In [5]:
def get_word(static_rec, dinamic_rec):
    if dinamic_rec == 'none' or dinamic_rec == 'static':
        return static_rec
    
    for index, row in actions_annotations_df.iterrows():
        if row['symbol'] == static_rec and row['action'] == dinamic_rec:
            return row['text']
    
    return static_rec

In [9]:
test_name = []
test_lable = []
count = 0

while len(test_name) < 100:
    rand = random.randint(1, 20400)
    
    if annotations_df.iloc[rand, 0] not in test_name and annotations_df.iloc[rand, 1] in actions:
        test_name.append(annotations_df.iloc[rand, 0])
        test_lable.append(annotations_df.iloc[rand, 1])

for i in range(100):
    actions_rows = annotations_df[annotations_df['attachment_id'] == test_name[i]]
    sequence = []
    static = []
    for index, row in actions_rows.iterrows():
        length = int(row['length'])
        # Путь к видеофайлу
        video_path = os.path.join(data_path, test_name[i] + '.mp4')

        # Проверка существования видеофайла
        if not os.path.exists(video_path):
            print("Видеофайл {} не найден".format(video_path))
            continue

        # Чтение видеофайла
        cap = cv2.VideoCapture(video_path)

        last_rec = None

        for frame_num in range(length):
            # Read frame
            ret, frame = cap.read()

            if ret == False:
                continue
                    
            # 10 кадров в начале обрезаются и обрезаются 5 последних кадров
            if frame_num >= 40 or frame_num < 10:
                continue
            
            static.append(static_recognition(frame))

            last_rec = pose_recognition(frame)
            sequence.append(last_rec)

        # дублирование последнего кадра при малом количестве кадров
        length_tmp = length
        while length_tmp < 40:
            sequence.append(last_rec)
            length_tmp += 1

        cap.release()
    
    counter = Counter(static)
    most_common_recognitions = counter.most_common(2)
    most_common_recognition = most_common_recognitions[0]

    if (len(most_common_recognitions) > 1 and most_common_recognition[0] == None):
        most_common_recognition = most_common_recognitions[1]

    if most_common_recognition[0] != None:
        dinamic_rec = dinamic_recognition(sequence)
        curr_sign = get_word(most_common_recognition[0], dinamic_rec)

    if curr_sign == test_lable[i]:
        count += 1

    print(f'{test_lable[i]} | {curr_sign}')

print(f'Accuracy: {count/100}')

1/1 [==============================] - 0s 17ms/step
Л | Л
1/1 [==============================] - 0s 17ms/step
Н | Н
1/1 [==============================] - 0s 18ms/step
ответить | ответить
1/1 [==============================] - 0s 19ms/step
два | два
1/1 [==============================] - 0s 22ms/step
день | день
1/1 [==============================] - 0s 17ms/step
И | И
1/1 [==============================] - 0s 23ms/step
спокойный | спокойный
1/1 [==============================] - 0s 16ms/step
Ь | Ь
1/1 [==============================] - 0s 16ms/step
опасный | опасный
1/1 [==============================] - 0s 16ms/step
день | день
1/1 [==============================] - 0s 16ms/step
хотеть | хотеть
1/1 [==============================] - 0s 17ms/step
завтра | хорошо
1/1 [==============================] - 0s 16ms/step
друг | друг
1/1 [==============================] - 0s 16ms/step
Т | Т
1/1 [==============================] - 0s 20ms/step
Ф | Ф
1/1 [==============================] - 0s 17ms